# **Analyse Exploratoire : Building Energy Benchmarking (Seattle)**

Le dataset utilisé est [téléchargeable ici](https://s3.eu-west-1.amazonaws.com/course.oc-static.com/projects/Data_Scientist_P4/2016_Building_Energy_Benchmarking.csv).
<br><br>
La définition des variables est [disponible ici](https://data.seattle.gov/dataset/2016-Building-Energy-Benchmarking/2bpz-gwpy).
<br><br>
*Le programme de mesure et de déclaration de la consommation d'énergie des bâtiments de Seattle (SMC 22.920) exige des propriétaires de bâtiments non résidentiels et multifamiliaux (de 20 000 pieds carrés ou plus) de suivre les performances énergétiques et de les déclarer chaque année à la ville de Seattle. Les bâtiments représentent 33 % des émissions principales de Seattle. La politique de mesure d'évaluation soutient les objectifs de Seattle visant à réduire la consommation d'énergie et les émissions de gaz à effet de serre des bâtiments existants. En 2013, la ville de Seattle a adopté un plan d'action climatique pour atteindre zéro émission nette de gaz à effet de serre d'ici 2050. La mesure, la déclaration annuelle et la divulgation des performances des bâtiments sont des éléments fondamentaux pour créer une valeur marchande plus élevée pour l'efficacité énergétique.*
<br><br>
**Ces données concernent uniquement l'année 2016.**